#### **Exercise 5.1**
##### What is the delay for the following types of 64-bit adders?
##### Assume that each two-input gate delay is 150 ps and that a full adder delay is 450 ps.

a) Ripple-Carry adder:

$t_{ripple} = 64(450ns) = 28.8us$

b) Carry-Lookahead adder:

$t_{CLA} = 150ns + 6(150ns) + (\frac{64}{4} -1)(150ns)2 + 4(450ns) = 7.35us$

$t_{pg}$ = time to compute $P_{i}$ and $G_{i}$

$t_{pg-block}$ = time to compute entire block - num of gates*delay of gate

$(\frac{N}{k}-1)t_{and-or}$ = number of blocks the carry should pass (before the last block) * AND/OR gate delay

$kt_{fa}$ = number of bits in a block * delay of full adder


#### **Exercise 5.3**
##### Explain why a designer might choose to use a ripple-carry adder instead of a carry-lookahead adder.

Ripple carry adder uses less two-input gates, which mean using it uses less space and power consumption.

#### **Exercise 5.8**
##### Design the following comparators for 32-bit unsigned numbers. Sketch the schematics.

a) not equal:

![alt text](5_8_a.png "Title")


b) greater than or equal to:

![alt text](5_8_b.png "Title")


c) less than:

![alt text](5_8_c.png "Title")


#### **Exercise 5.9**
#### a) Give an example of two 4-bit signed numbers A and B for which a 4-bit signed comparator correctly computes A < B ####

$A = 2_{10} = 0010_{2}$ , $B = 3_{10} = 0011_{2}$

$A+(B') = 2_{10} - 3_{10} = 0010 + 1101 = 1111 = -1_{10}$

$N_{Flag} = 1, V_{Flag} = 0$

***Correct***

#### b) Give an example of two 4-bit signed numbers A and B for which a 4-bit signed comparator incorrectly computes A < B

$A = -7_{10} = 1001_{2}$ , $B = 6_{10} = 0110_{2}$


$A+(B') = -7_{10} - 6_{10} = 1001 + 1010 = 1_0011 = 3_{10}$

$N_{Flag} = 0, V_{Flag} = 1$

***Wrong***

#### c) In general, when does the N-bit signed comparator operate incorrectly?

The N-bit signed comperator operate incorrectly when there is a overflow from the MSB.

This will happen when we compare integer with the oppsite sign - positive to negative or negative to positive

#### **Exercise 5.10**
#### Modify the N-bit signed comparator of Figure 5.12 to correctly compute A < B for all N-bit signed inputs A and B.####

```
Fixed Comperator

module comparators #(parameter N = 8)
                    (input  logic signed [N-1:0] a, b,
                    output logic eq, neq, lt, lte, gt, gte);

    // we will need to handle the case for overflow/underflow when comparing signed numbers:
    //if both a and b are positive or both are negative, we can compare them directly.

    logic signed [N-1:0] diff;
    logic v; // Overflow flag
    logic n; // Negative flag

    assign diff = a - b;
    assign n = diff[N-1];
    assign v = (a[N-1] != b[N-1]) && (diff[N-1] != a[N-1]);

    assign eq = (a == b);
    assign neq = (a != b);
    assign lt = (n ^ v); // a < b if the result is negative and there is no overflow, or if there is an overflow and the result is positive
    assign lte = (lt | eq);
    assign gt = (!lte);
    assign gte = (!lt);
endmodule

```

#### **Exercise 5.11** ####
#### Design the 32-bit ALU shown in Figure 5.15 using your favorite HDL. ###
#### You can make the top-level module either behavioral or structural. ####

```
32 bit ALU:

module ALU_32b #(parameter N = 32)
            (input  logic [N-1:0] a, b,
            input  logic [1:0] ctrl,
            output logic [N-1:0] result
);

    logic [N-1:0] b_inv;
    logic [N-1:0] b_final;
    logic [N-1:0] sum;

    assign b_inv   = ~b;
    
    assign b_final = ctrl[0] ? b_inv : b;

    // if ctrl[0] is 1, we want to perform subtraction, which is a + (~b) + 1
    // if ctrl[0] is 0, we want to perform addition, which is a + b + 0
    assign sum     = a + b_final + ctrl[0];

    always_comb begin
        case (ctrl)
            2'b00: result = sum;
            2'b01: result = sum; 
            2'b10: result = a & b; 
            2'b11: result = a | b; 
            default: result = {N{1'bx}}; 
        endcase
    end
    
endmodule

```